In [1]:
using Pkg
using Knet: Knet, param, param0, @diff, grad,RNN,mat, params, KnetArray, conv4, Data, relu, pool, dropout
using Random: shuffle!
using IterTools: ncycle
using TestImages,Images, ImageView
using PyCall
@pyimport pickle
@pyimport numpy

┌ Info: Recompiling stale cache file /home/okolukisa/.julia/compiled/v1.2/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /home/okolukisa/.julia/compiled/v1.2/Images/H8Vxc.ji for Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /home/okolukisa/.julia/compiled/v1.2/ImageView/4mtgY.ji for ImageView [86fae568-95e7-573e-a6b2-d8a6b900c9ef]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /home/okolukisa/.julia/compiled/v1.2/PyCall/GkzkC.ji for PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1240
┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:407
└ @ PyCall /home/okolukisa/.julia/packages/PyCall/ttONZ/src/PyCall.jl:407


In [2]:
# @doc conv4

In [3]:
# x = randn(Float32, 9,9,1,1)

In [4]:
# c = conv4(4,4,1,3)
# c(x)

In [44]:
word_to_ix = []
label_to_ix = []

instructions = Tuple{Array{Int64,1},Int64}[]
# labels = []
# sentence = []
# words = []

wdict = Dict()
tdict = Dict()
w2i(x) = get!(wdict, x, 1+length(wdict))
t2i(x) = get!(tdict, x, 1+length(tdict))
UNK = w2i("<unk>")

1

In [60]:
open("instructions.txt") do f
    for line in eachline(f)
#         line = split(strip(line),",")
#         label = [line[2]]
#         sentence = map(x-> lowercase(x),split(strip(line[1])," "))
#         append!(instructions,(sentence,label))
        sentence, label = split(strip(lowercase(line)), ",")
        wordids = w2i.(split(sentence))
        labelid = t2i(label)
        println(labelid," ",sentence )
        push!(instructions, (wordids, labelid))
#         push!(words, split(strip(sentence), " "))
    end
#      print(instructions)
#     print(words)
end


1 climb down the ladder
1 climb down the ladder
1 climb down the ladder
1 climb down the ladder
1 climb down the ladder
1 climb up the ladder
1 climb up the ladder
1 climb up the ladder
1 climb up the ladder
2 get the key
2 get the key
3 get the sword
4 get the torch
5 go between the lasers
5 go between the lasers
5 go between the lasers
5 go between the lasers
6 go to the bottom of the room
6 go to the bottom of the room
6 go to the bottom of the room
6 go to the bottom of the room
6 go to the bottom of the room
6 go to the bottom room
6 go to the bottom room
6 go to the bottom room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the center of the room
6 go to the left room
6 go to the lef

In [7]:
# function prepare_sentence(sent, to_ix)
#     sent = split(strip(lowercase(sent))," ")
#     idxs = [to_ix[w] for w in sent]
#     return torch.tensor(idxs)
# end

In [8]:
EMBEDDING_DIM = 20
HIDDEN_DIM_LSTM = 10
VOCAB_SIZE = length(word_to_ix)
LABEL_SIZE = length(label_to_ix)

# text_model = LSTMClassifier()
# image_model = ConvNetClassifier()
# loss_function = nn.CosineEmbeddingLoss()
# optimizer1 = optim.SGD(text_model.parameters(), lr = 0.001)
# optimizer2 = optim.SGD(image_model.parameters(), lr = 0.001)

0

In [9]:
using PyCall
@pyimport pickle

function mypickle(filename, obj)
    out = open(filename,"w")
    pickle.dump(obj, out)
    close(out)
 end

function myunpickle(filename)
    r = nothing
    @pywith pybuiltin("open")(filename,"rb") as f begin
        r = pickle.load(f)
    end
    return r
end

myunpickle (generic function with 1 method)

In [10]:
l = myunpickle("dataset/dataset_true.pickle")

wdict2 = Dict()
tdict2 = Dict()
w2i2(x) = get!(wdict2, x, 1+length(wdict2))
t2i2(x) = get!(tdict2, x, 1+length(tdict2))
UNK2 = w2i2("<unk>")

1

In [11]:
# function GetPreProcessedImageArray(img, cropfromtop::Float64, cropfrombottom::Float64, target_height::Int, target_width::Int)
#     img = img ./ 255
#     height, width, channels = size(img)
#     state_image = permutedims(img,(1,2,3))
#     state_image2 = (state_image[:,Int(height*cropfromtop)+1:Int((1-cropfrombottom)*height),:])
#     grayscale = Gray.(state_image2)
# #     reshape(Float64.(imresize(grayscale, (1,target_height, target_width))), (target_width,target_height))
# end

In [12]:
images = []
im_sentences = []
for i in 1:length(l)
    (f1,f2), sent = l[i]
    #both = numpy.hstack((f1,f2))
    # append!(images,split(strip(sent)," "))
    #resf1 = permutedims((f1[:,:,:])/255,(3,1,2))
    #imshow(resf1)
    #resf2 = permutedims((f1[:,:,:])/255,(3,1,2))
    both = hcat(f1,f2)
#     tst = GetPreProcessedImageArray(both, 0.0, 0.0, 210,320)
#     println(size(tst))
#     imshow(tst)
#     break
    
#     grayf1 = Gray.(resf1)
#     grayf2 = Gray.(resf2)
#     donef1 = reshape(Float64.(imresize(grayf1, (1,210, 60))), 210*60)
#     donef2 = reshape(Float64.(imresize(grayf2, (1,210, 60))), 210*60)
#     both = hcat(donef1,donef2)
#     both = hcat(grayf1,grayf2)
#     print(size(grayf1))
  

#     When we change it to (210,320) it will work better I guess, now its just changed to Float64
    b3 = reshape(Float32.(both), (210,320,3,1))
#         println(typeof(b3))
    
    wordids = w2i2.(split(sent))
    #labelid = t2i(label)
    push!(im_sentences, wordids)
    push!(images,b3)
end
# println(images)

In [13]:
typeof(images[1])

Array{Float32,4}

In [14]:
# Define a convolutional layer:
struct Conv; w; b; f; p; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, dropout(x,c.p)) .+ c.b))
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [15]:
struct Convp; w; b; f; p; end
(c::Convp)(x) = c.f.(conv4(c.w, dropout(x,c.p)) .+ c.b)
Convp(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Convp(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Convp

In [16]:
# Redefine dense layer (See mlp.ipynb):
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [17]:
struct Densep; w; b; f; p; end
(d::Densep)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Densep(i::Int,o::Int,f=relu;pdrop=0) = Densep(param(o,i), param0(o), f, pdrop)

Densep

In [18]:
# Let's define a chain of layers
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d)

In [19]:
#Lets get a try in

testChain = Chain(
    Conv(5,5,3,32),#103*158
    Conv(5,5,32,32),#49*77
    Conv(4,4,32,64),#23*37
    Convp(3,3,64,64),#21*35*64 = 47040
    Dense(47040,10,pdrop=0.3))
summary.(l.w for l in testChain.layers)


5-element Array{String,1}:
 "5×5×3×32 AutoGrad.Param{Array{Float32,4}}" 
 "5×5×32×32 AutoGrad.Param{Array{Float32,4}}"
 "4×4×32×64 AutoGrad.Param{Array{Float32,4}}"
 "3×3×64×64 AutoGrad.Param{Array{Float32,4}}"
 "10×47040 AutoGrad.Param{Array{Float32,2}}" 

In [20]:
agent_embedding= []
for i in 1:347
 push!(agent_embedding, testChain(images[i]))
end


In [21]:
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]

In [63]:
embedlayer = Embed(length(instructions),128)
# typeof(instructions)

allw = vcat((x->x[1]).(instructions)...)
allt = vcat((x->x[2]).(instructions)...)
N = length(allw) ÷ 2
B = 2
allw = permutedims(reshape(allw[1:N*B],N,B))
# allt = permutedims(reshape(allt[1:N*B],N,B))
# d = []
allw[1]

2

In [61]:
BATCHSIZE = 2
SEQLENGTH = 1;

function seqbatch(x,y,B,T)
    N = length(y) ÷ B
    x = permutedims(reshape(x[1:N*B],N,B))
    y = permutedims(reshape(y[1:N*B],N,B))
    d = []; for i in 0:T:N-T
        push!(d, (x[:,i+1:i+T], y[:,i+1:i+T]))
    end
    return d
end
allw = vcat((x->x[1]).(instructions)...)
allt = vcat((x->x[2]).(instructions)...)
d = seqbatch(allw, allt, BATCHSIZE, SEQLENGTH);


In [24]:
# A LSTM implementation in Knet

function lstm(param, state, input)
    weight,bias = param
    hidden,cell = state
    h       = size(hidden,2)
    gates   = hcat(input,hidden) * weight .+ bias
    forget  = sigm.(gates[:,1:h])
    ingate  = sigm.(gates[:,1+h:2h])
    outgate = sigm.(gates[:,1+2h:3h])
    change  = tanh.(gates[:,1+3h:4h])
    cell    = cell .* forget + ingate .* change
    hidden  = outgate .* tanh.(cell)
    return (hidden,cell)
end;

In [25]:
# For running experiments we will use the Adam algorithm which typically converges faster than SGD.
function trainresults(file,maker,savemodel)
    if (print("Train from scratch? "); readline()[1]=='y')
        model = maker()
        results = ((nll(model,dtst), zeroone(model,dtst))
                   for x in takenth(progress(adam(model,ncycle(dtrn,5))),100))
        results = reshape(collect(Float32,flatten(results)),(2,:))
        Knet.save(file,"model",(savemodel ? model : nothing),"results",results)
        Knet.gc() # To save gpu memory
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        model,results = Knet.load(file,"model","results")
    end
    println(minimum(results,dims=2))
    return model,results
end

trainresults (generic function with 1 method)

In [26]:
VOCABSIZE = length(instructions)
EMBEDSIZE = 128
HIDDENSIZE = 128
OUTPUTSIZE = length(instructions);

In [27]:
t2maker() = Tagger(VOCABSIZE,EMBEDSIZE,HIDDENSIZE,OUTPUTSIZE)
(t2,r2) = trainresults("tagger113c.jld2",t2maker,true);

Train from scratch? stdin> n


Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /buildworker/worker/package_linux64/build/src/locks.h:166 [inlined]
jl_typeinf_end at /buildworker/worker/package_linux64/build/src/gf.c:2595
typeinf_ext at ./compiler/typeinfer.jl:569
typeinf_ext at ./compiler/typeinfer.jl:599
jfptr_typeinf_ext_1.clone_1 at /home/okolukisa/Downloads/julia-1.2.0-linux-x86_64/julia-1.2.0/lib/julia/sys.so (unknown line)
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2191
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1614 [inlined]
jl_type_infer at /buildworker/worker/package_linux64/build/src/gf.c:207
jl_compile_method_internal at /buildworker/worker/package_linux64/build/src/gf.c:1773
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2196
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1614 [inlined]
jl_f__apply at /buildworker/worker/package_linux64/build/src/builtins.c:563
j

EOFError: EOFError: read end of file

In [28]:
# struct LSTMClas
#     embeddings
#     lstm
#     fullyconnected
#     hidden
# end

# function LSTMClas()
#     embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM) 
#     lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM_LSTM)
#     fullyconnected = nn.Linear(HIDDEN_DIM_LSTM, 10)
# end
# #NOt done


In [29]:
# class LSTMClassifier(nn.Module):

# 	def __init__(self):
		
# 		super(LSTMClassifier, self).__init__()

# 		self.embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
# 		self.lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM_LSTM)
# 		self.fullyconnected = nn.Linear(HIDDEN_DIM_LSTM, 10)
# 		self.hidden = self.init_hidden()

# 	def init_hidden(self):
# 		# the first is the hidden h
# 		# the second is the cell  c
# 		return (autograd.Variable(torch.zeros(1, 1, HIDDEN_DIM_LSTM)),
#                 autograd.Variable(torch.zeros(1, 1, HIDDEN_DIM_LSTM)))

# 	def forward(self, sentence):

# 		embeds = self.embeddings(sentence)
# 		x = embeds.view(len(sentence), 1, -1)
# 		lstm_out, self.hidden = self.lstm(x, self.hidden)
# 		#print (lstm_out)
# 		y  = self.fullyconnected(lstm_out[-1])
# 		# log_probs = F.log_softmax(y)
# 		#print (y)
# 		return y
